<a href="https://colab.research.google.com/github/qahaidari/Keras-classification-with-finetuned-vgg16-flask-deployment/blob/main/classification_vgg16_finetuned_backend_flask_singleImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In this project, we want to use the [fine-tuned VGG16 model](https://github.com/qahaidari/Keras-classification-with-finetuned-vgg16-flask-deployment/blob/main/DogsvsCats_vgg16_finetuned.ipynb) to do predictions on images of cats and dogs. The VGG16 model is already fine-tuned to predict on two classes of cats and dogs. Then the model will be deployed on Flask. A Flask web service for the backend of the project and a Flask web application for the frontend of the project will be implemented. In the frontend, the user can upload an image of a cat or a dog and send it to the backend web service, where the fine-tuned VGG16 model will receive the image, do a prediction and return the result to the user. Because the aim is to use VGG16 neural network to predict on images of cats and dogs, the Flask services for building frontend and backend parts of this application are implemented on Google Colab so that we can use the Colab GPU.

For running Flask on Colab, flask-ngrok should be installed; [tutorial](https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b).

In [2]:
!pip install flask-ngrok

Below is the implementation of the backend for a simple application where user sends some data from the client side as a json file and the backend receives it and sends back a response after processing.

In [3]:
# a simple backend with flask for returning what the user types on frontend
from flask_ngrok import run_with_ngrok
from flask import request
from flask import jsonify
from flask import Flask

app = Flask(__name__)
run_with_ngrok(app)
@app.route('/',methods=['POST'])
def hello():
    message = request.get_json(force=True)
    name = message['name']
    response = {
        'greeting': 'Hello, ' + name + '!'
    }
    return jsonify(response)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://006e48e16b29.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


Now let's develop a backend web service where predictions of images of cats and dogs are implemented using a fine-tuned VGG16 model. The fine-tuned VGG16 model is available in the same directory in the form of a h5 file.

In [4]:
import base64
import numpy as np
import io
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from flask import request
from flask import jsonify
from flask import Flask
from flask_ngrok import run_with_ngrok

In [5]:
app = Flask(__name__)

We define a function to load our VGG16 fine-tuned h5 model.

In [6]:
# function to load h5 file of the VGG16 fine-tuned model that we had saved already
def get_model():
    global model
    model = load_model('/content/drive/MyDrive/dogs-vs-cats/VGG16_cats_and_dogs.h5')
    print(" * Model loaded!")

In [7]:
# this function preprocesses the image so that it can be passed to our model
def preprocess_image(image, target_size):
    if image.mode != "RGB":
        image = image.convert("RGB")
    image = image.resize(target_size)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    return image

In [8]:
print(" * Loading Keras model...")
get_model()

 * Loading Keras model...
 * Model loaded!


In [9]:
run_with_ngrok(app)
@app.route("/", methods=["POST"])
def predict():
    message = request.get_json(force=True)
    encoded = message['image']
    decoded = base64.b64decode(encoded)
    image = Image.open(io.BytesIO(decoded))
    processed_image = preprocess_image(image, target_size=(224, 224))

    prediction = model.predict(processed_image).tolist()

    response = {
        'prediction': {
            'cat': prediction[0][0],
            'dog': prediction[0][1]
        }
    }
    return jsonify(response)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ba8cf2cc7cdf.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [13/Apr/2021 22:21:42] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 22:28:38] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2021 22:28:49] "POST / HTTP/1.1" 200 -
